In [1]:
%cd ..

/home/prayson/Code/hadithi/dev.io


In [172]:
from pathlib import Path
from zipfile import ZipFile

from httpx import AsyncClient, Client
from mimesis import Person
from mimesis.locales import Locale
import jmespath
import pandas as pd

In [89]:
MIMESIS_LOCALE_URL = "https://mimesis.name/en/master/locales.html#overriding-locale"
COUNTRY_CODE_CSV = "assets/languages.csv"
[language_codes, *_] = pd.read_html(MIMESIS_LOCALE_URL)
country_codes = pd.read_csv(COUNTRY_CODE_CSV, usecols=["Country", "Two Letter"])

In [110]:
country_iso = (
    country_codes
     .transform(lambda d: d.str.lower())
     .rename(columns={"Country": "country", "Two Letter":"code"})
     #.drop_duplicates(subset=["country"], keep="first")
     .assign(code = lambda d: d["code"].str.split("-").str[0])
     .query("code != 'en'")
)

In [111]:
country_language = (language_codes[["Code", "Name"]]
     .transform(lambda d: d.str.lower())
     .merge(country_iso, how="left", left_on="Code", right_on="code")
     .dropna(subset=["country", "code"])
     .loc[:, ["code", "country"]]
     .set_index("country")
     .to_dict()
     .get("code")

)

In [5]:
DATA_DIR = Path("assets/books.zip")

with ZipFile(DATA_DIR) as z:
    print(z.namelist())
    files = (file for file in z.namelist() if Path(file).suffix == ".csv")
    data = {Path(file).stem.lower(): pd.read_csv(z.extract(file),low_memory=False) for file in files}     

['Books.csv', 'Ratings.csv', 'Users.csv', 'recsys_taxonomy2.png']


In [140]:

def get_name(language:str) -> str:
    person = Person(language)
    return person.full_name()


In [156]:
users = pd.read_csv("assets/users.csv", dtype_backend="pyarrow", dtype={"age":"Int64"})

In [157]:
users

,user_id,fake_name,location,age
0,1,Catrice Riggs,"nyc, new york, usa",<NA>
1,2,Johnson Colon,"stockton, california, usa",18
2,3,Лолита Кузьмин,"moscow, yukon territory, russia",<NA>
3,4,Abraim Melo,"porto, v.n.gaia, portugal",17
4,5,Barton King,"farnborough, hants, united kingdom",<NA>
...,...,...,...,...
278853,278854,Renaldo Gomez,"portland, oregon, usa",<NA>
278854,278855,Elizbeth Bird,"tacoma, washington, united kingdom",50
278855,278856,Arda Guichard,"brampton, ontario, canada",<NA>
278856,278857,Clifford Erickson,"knoxville, tennessee, usa",<NA>


In [271]:
client = Client(base_url="https://openlibrary.org/api")
ISBN = "0091906814" #"0395974682" lord of the rings  #"0140301690" - Alice in Wonderland #ISBN=ASIN?
r = client.get(url="/books", params={"bibkeys": f"ISBN:{ISBN}", "format":"json"}, timeout=10)

In [272]:
b = r.json()
b

{'ISBN:0091906814': {'bib_key': 'ISBN:0091906814',
  'info_url': 'https://openlibrary.org/books/OL7315585M/How_to_Win_Friends_and_Influence_People',
  'preview': 'noview',
  'preview_url': 'https://openlibrary.org/books/OL7315585M/How_to_Win_Friends_and_Influence_People',
  'thumbnail_url': 'https://covers.openlibrary.org/b/id/69810-S.jpg'}}

In [248]:
open_client = Client(base_url="https://openlibrary.org")

In [273]:
[url, *_] = jmespath.search("*.info_url", b)
work = url.split("/")[-2]

r = open_client.get(f"/books/{work}.json", timeout=10)
r

<Response [200 OK]>

In [274]:
r.json()

{'publishers': ['Vermilion'],
 'identifiers': {'goodreads': ['33288'], 'librarything': ['68250']},
 'source_records': ['bwb:9780091906818'],
 'title': 'How to Win Friends and Influence People',
 'isbn_10': ['0091906814'],
 'number_of_pages': 304,
 'covers': [69810],
 'isbn_13': ['9780091906818'],
 'lc_classifications': ['BF636'],
 'publish_date': 'April 5, 2007',
 'key': '/books/OL7315585M',
 'authors': [{'key': '/authors/OL107569A'}],
 'oclc_numbers': ['62795939'],
 'works': [{'key': '/works/OL1063301W'}],
 'type': {'key': '/type/edition'},
 'latest_revision': 11,
 'revision': 11,
 'created': {'type': '/type/datetime', 'value': '2008-04-29T13:35:46.876380'},
 'last_modified': {'type': '/type/datetime',
  'value': '2021-05-04T11:01:27.515271'}}

In [275]:
b = r.json()

g = open_client.get(url=f'{b.get("works")[0].get("key")}.json', timeout=10)
g

<Response [200 OK]>

In [276]:
g.json()

{'title': 'How to Win Friends and Influence People',
 'covers': [12973211,
  6982482,
  8231858,
  1334554,
  2295044,
  4988481,
  2295570,
  7942941,
  8770342,
  9336127,
  10653631,
  10715296,
  10764191,
  10829208,
  407939,
  5317691,
  404729,
  8599963,
  489942,
  8329393,
  8332729,
  8571360,
  9162292,
  10162496,
  6882239,
  6888174,
  10465148,
  11332751,
  11425532,
  9247891,
  8606048,
  8741212,
  11932305,
  9019798,
  9155245,
  9245280,
  9299832,
  11409754,
  11964323,
  13843827,
  12875128,
  12883424,
  13112610],
 'key': '/works/OL1063301W',
 'authors': [{'author': {'key': '/authors/OL107569A'},
   'type': {'key': '/type/author_role'}}],
 'type': {'key': '/type/work'},
 'description': {'type': '/type/text',
  'value': 'Humans are relational beings. This is the best self-improvement book to know how to create meaningful and fruitful relationships.\r\nFor more than sixty years the rock-solid, time-tested advice in this book has carried thousands of now famo

In [158]:
# https://openlibrary.org/works/OL18020194W/bookshelves.json
# https://openlibrary.org/works/OL18020194W/ratings.json

In [277]:
open_client.get(url=f'{jmespath.search("works[0].key", b)}/bookshelves.json', timeout=10).json()

{'counts': {'want_to_read': 5498,
  'currently_reading': 508,
  'already_read': 216}}

In [278]:
open_client.get(url=f'{jmespath.search("works[0].key", b)}/ratings.json', timeout=10).json()

{'summary': {'average': 4.259067357512953,
  'count': 193,
  'sortable': 4.088984102582939},
 'counts': {'1': 12, '2': 6, '3': 21, '4': 35, '5': 119}}

In [279]:
jmespath.search("authors[0].author.key", g.json())

'/authors/OL107569A'

In [280]:
open_client.get(url=f'{jmespath.search("authors[0].author.key", g.json())}.json', timeout=10).json()

{'title': '(Carnegie, Dale, 1888-1955)',
 'key': '/authors/OL107569A',
 'name': 'Dale Carnegie',
 'type': {'key': '/type/author'},
 'death_date': 'November 1, 1955',
 'photos': [9160344, 7353715, 6732648],
 'personal_name': 'Dale Carnegie',
 'alternate_names': ['DALE CARNEGIE',
  'Dale carnegie',
  'Ka na ji (Carnegie, Dale, 1888-1955)',
  'Dale Breckenridge Carnegie',
  'Qia nai ji (Carnegie, Dale, 1888-1955)',
  'Dale CARNEGIE',
  'Ka nai ji (Carnegie, Dale',
  'dale carnegie',
  'Carnegie, Dale, 1888-1955.',
  'D. Carnegie',
  '(mei) Kanaiji (Carnegie, Dale)',
  'CARNEGIE DALE',
  'Dale Dale Carnegie',
  'CARNEGIE,DALE',
  'Carnegie, Dale, 1st',
  '(mei) Ka, Neiji (Carnegie, Dale',
  'Ka nei ji (Carnegie, Dale)',
  'Ka nei ji',
  'Carnegie, Dale',
  'dale-carnegie',
  'Carnegie Dale'],
 'birth_date': 'November 24, 1888',
 'source_records': ['ia:renxingderuodian0000meik',
  'amazon:2764023065',
  'amazon:9386037750',
  'amazon:6055092220',
  'bwb:9781519035554',
  'amazon:9350881241'

In [220]:
data["books"].sort_values("Year-Of-Publication")

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
212233,0947901051,Collect Post Office Cards,A. Roberts,0,Benham (A. Buckingham) Ltd,http://images.amazon.com/images/P/0947901051.0...,http://images.amazon.com/images/P/0947901051.0...,http://images.amazon.com/images/P/0947901051.0...
51621,0006479561,Sacred Clowns,Tony Hillerman,0,Fairmount Books Ltd Remainders,http://images.amazon.com/images/P/0006479561.0...,http://images.amazon.com/images/P/0006479561.0...,http://images.amazon.com/images/P/0006479561.0...
193438,0140382690,The Enemy You Killed,Peter McFarlane,0,Penguin Books Australia Ltd,http://images.amazon.com/images/P/0140382690.0...,http://images.amazon.com/images/P/0140382690.0...,http://images.amazon.com/images/P/0140382690.0...
25016,0879972734,Walkers On the Sky,David J Lake,0,Penguin Putnam~mass,http://images.amazon.com/images/P/0879972734.0...,http://images.amazon.com/images/P/0879972734.0...,http://images.amazon.com/images/P/0879972734.0...
17668,0590023888,Misty of Chincoteague,Marguerite Henry,0,Scholastic Inc,http://images.amazon.com/images/P/0590023888.0...,http://images.amazon.com/images/P/0590023888.0...,http://images.amazon.com/images/P/0590023888.0...
...,...,...,...,...,...,...,...,...
80264,0140301690,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...
97826,0140201092,Outline of European Architecture (Pelican S.),Nikolaus Pevsner,2050,Penguin USA,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN
